# Семинар 11. Подбор гиперпараметров. Байесовская оптимизация. Optuna 🔍

## Как валидируем

- Параметры нейронной сети обучаемые, учим градиентным спуском
- Гиперпараметры - подбираем сами, глядя на графики обучения или из каких-либо других соображений
  - **hold-out**: делим датасет на 3 части, по валидации судим, как изменить `lr` и другие гиперпараметры
  - **k-fold**: кросс-валидация, до сих пор не использовали, но так тоже можно (занимает больше времени, поэтому не используется на больших датасетах)

![](images/0.png)

*Hold-out: делим датасет на 3 части (train, val, test). Учимся на train, по val выбираем гиперпараметры (хотим, чтобы модель на val давала максимальную метрику), на test проверяем готовую обученную модель*

![](images/9.png)

*K-fold: делим тренировочные данные на k частей (фолдов), и в цикле по k итерациям учимся на k-1 фолде, валидируясь на оставшемся 1. У нас получится k разных вариантов одной и той же модели, которые отличаются только теми данными, на которых они учились. Это позволяет надежнее оценивать качество модели*

## Как подбираем гиперпараметры

1. Мы можем написать функцию потерь для своей задачи, значение которой будет зависеть от выходов модели, а значит и от её весов. Так как вся наша система дифференцируемая, то мы можем учить модель при помощи градиентного спуска
2. Хотелось бы то же самое делать с гиперпараметрами (чтобы у нас была $f(x) \sim Accuracy(x)$, где $x$ - гиперпараметры), но не можем. Почему?
  - Вычисление $f(x)$ очень дорогое - нужно обучить модель и посчитать её метрику
  - Непонятно, как брать градиент по гиперпараметрам - в случае параметров есть понятная зависимость, а в случае гиперпараметров как будто бы нет (вот попробуйте написать в виде формулы зависимость `Accuracy` от `warmup_epochs` и градиент от этого посчитать...)
  - Считать градиент численно тоже плохая мысль, потому что вычислять $f(x)$ дорого
  - Непонятно, что делать, если гиперпараметр дискретный
  - Нужна обязательно гладкая функция, чтобы можно было её дифференцировать, и не любая метрика подойдёт

Что делать?

## Grid search

- Выбираем для каждого гиперпараметра несколько значений
- Составляем все комбинации и перебираем их
- На каждой из них обучаем и тестируем
- Выбираем лучшую

## Random search

 - Качество модели может от одного гиперпараметра зависеть сильно, а от других гиперпараметров почти не зависеть
 - `Random Search` может за то же число итераций, что и `Grid Search`, рассмотреть более разнообразные значения гиперпараметров. Тем самым он с большей вероятностью найдёт те значения, которые больше всего влияют на качество модели, а значит, с большей вероятностью найдёт наилучшую комбинацию значений гиперпараметров.

 На рисунке ниже: `Random Search` за те же 9 итераций выбирает больше различных значений гиперпараметров и за счёт этого быстрее находит максимум

![](images/10.png)

Оба из этих подходов (`GridSearch` и `RandomSearch`) не очень эффективны, так как занимают много времени в случае большого числа гиперпараметров

## Байесовская оптимизация


Нередко оказывается, что человек может подобрать лучшие гиперпараметры, чем `GridSearch` и `RandomSearch`. Например, человек может потихоньку увеличивать некий гиперпараметр и смотреть на то, что происходит с целевой метрикой. Благодаря своему опыту и интуиции он сможет подобрать достаточно хорошее значение. В примере ниже дата-сайентист увеличил число нейронов в скрытом слое сначала до 100, а потом до 200. В первом случае метрика сильно возросла, а во втором изменилась всего на 2%. Кажется, что увеличивать на 100 снова уже не имеет смысла.

Главный недостаток `GridSearch` и `RandomSearch` - они **не учитывают предыдущую историю**, в отличие от дата-сайентиста, который подбирает "руками" гиперпараметры из каких-то своих соображений.

![](images/1.png)

При выборе гиперпараметров нам хотелось бы найти баланс между двумя стратегиями:
  - **exploration**, т.е. исследованием тех значений гиперпараметров, какие мы ещё не пробовали
  - **exploitation**, т.е. выбором семплов в тех областях, которые мы неплохо изучили и где у нас, как мы  считаем, хорошая метрика

### Идея

Как мы уже обсуждали, нам бы хотелось, чтобы у нас была некая функция $f(x)$, с помощью которой мы могли бы искать гиперпараметры. Но вычисление нашей $f(x)$ дорогое. Хотим найти оптимальные $x$, как можно реже вычисляя функцию $f$ в конкретной точке.

**Байесовская оптимизация** — это итерационный метод, позволяющий оценить оптимум функции, не дифференцируя её.

- На каждой итерации метод указывает, в какой следующей точке мы с наибольшей вероятностью улучшим нашу текущую оценку оптимума.
- Это позволяет значительно сократить количество вычислений функции, каждое из которых может быть довольно затратным по времени.

Мы можем сформулировать задачу поиска гиперпараметров для нейросети в терминах байесовской оптимизации:
- Наша функция - это значение валидационных метрик в зависимости от текущих гиперпараметров.
- Её вычисление затратно по времени - надо натренировать модель и провалидировать её
- Мы не можем вычислить градиенты этой функции по гиперпараметрам

Байесовская оптимизация имеет **две основные компоненты**:
- **Вероятностную модель**, которая аппроксимирует целевую функцию (метрику) в зависимости от исторических данных (всех предыдущих наборов гиперпараметров) 
- **acquisition функцию**, которая по некоторым статистикам вероятностной модели функции $f$ будет выбирать, в какой точке нужно вычислить значение $f$. Она должна балансировать между *exploration* и *exploitation*:
  - *exploration* - исследовать те точки, в которых дисперсия нашей вероятностной модели велика
  - *exploitation* - исследовать подробнее те точки, вблизи которых наша модель уже получала хороший результат

### Вероятностная модель

В качестве вероятностной модели часто выбирают **гауссовские процессы**.

Опеределение из Википедии:

> Случайный процесс с непрерывным временем является гауссовским тогда и только тогда, когда для любого конечного множества индексов $t_1, \ldots , t_k$ из множества индексов $T$: $X_{t_1, \ldots, t_k} = (X_{t_1}, \ldots, X_{t_k})$ является многомерной гауссовской случайной величиной

![](images/2.png)

[Пример гауссовских процессов (интерактивный)](http://smlbook.org/GP/)

Почему гауссовские процессы?

- У нас всегда есть элемент случайности (порядок батчей в даталоадере, случайная инициализация весов, аугментации)
- Когда мы запускаем обучение модели с разными сидами, то метрика у нас получается достаточно близкой от эксперимента к эксперименту
- Считаем, что эта величина случайная
- Считаем, что это случайная гауссова величина - другие мы и не умеем считать 🙃

У нас есть некая выборка $(X, y)$ - например, один гиперпараметр $X$ и метрика $y$, полученная с этим гиперпараметром, в 4 точках (обозначены крестиками). Мы можем аппроксимировать весь набор данных при помощи одной вероятностной модели (синяя линия) и посмотреть на то, где будет минимум (или максимум, смотря что мы ищем) у такой функции. Потом можем посмотреть на то, что получится с другой вероятностной моделью - где теперь окажется минимум? Если мы возьмем очень много вероятностных моделей, то у нас получится целая гистограмма

Как мы получаем кривые ниже, которыми аппроксимируем функции? Да мы просто генерируем их (делаем что-то типа `linspace` по оси $x$ и в каждой из точек генерируем значение из гауссового распределения)

![](images/3.png)

### Acquisition function

Хороший пример **acquisition function**: $\alpha(x) = \mu(x) + β\sigma(x)$. Здесь $\mu(x)$ - это среднее по вероятностным моделям, а $\sigma(x)$ - это дисперсия по ним. В точках, где у нас были сделаны измерения, мы считаем, что значение известно точно (дисперсия = 0).

 Если мы ищем максимум, то максимизируя такую $\alpha(x)$ мы будем следовать как принципам *exploration* (большая $\sigma(x)$ внесёт свой вклад), так и *exploitation* (если $\mu(x)$ большое, то это даст свой вклад в $\alpha(x)$)

График такой $\alpha(x)$:

![](images/4.png)

На каждой итерации смотрим на то, где у нас максимум $\alpha(x)$ (серая кривая), выбираем новую точку для того, чтобы с этими гиперпараметрами обучить сетку и проверить метрику

![](images/5.png)

[Пример ноутбука с кодом](https://colab.research.google.com/github/krasserm/bayesian-machine-learning/blob/master/bayesian_optimization.ipynb)

- Можно выбирать разные функции в качестве $\alpha(x)$ (см. картинки ниже)
- Можем использовать некоторые трюки для того, чтобы обходиться с дискретными гиперпараметрами

![](images/6.png)

### Резюме по байесковской оптимизации

Мы заменяем нерешаемую задачу поиска $x_{min} = \underset{x⊂𝒳}{argmin}\ f(x)$ на решаемую задачу $x_{t+1} = \underset{x⊂𝒳}{argmax}\ \alpha(x|S_t)$, которая будет при большом числе итераций сходиться к исходной

- Не так сложно вычислять $\alpha(x)$
- Градиенты $\alpha(x)$ обычно имеются
- Всё ещё нужно уметь считать $x_{t+1}$ - градиентный метод и т.п.

## Optuna

**Оптюна** состоит из

- **Сэмплера** - он выбирает примеры из пространства гиперпараметров. Будем считать, что мы используем TPE sampler. Идея у TPE похожа на то, что было описано выше, а в [подробности](https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f) мы вникать не будем
- **Прунера** - на каждой эпохе он решает, есть ли смысл обучать модель с данным набором гиперпараметров. О том, как именно работает прунер, мы тоже не будем говорить 😿





### Примеры попроще

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Пример №0: как вообще это выглядит

```python
import optuna

def objective(trial):  # `trial` is an object passed by Optuna.
    some_machine_learning_logic(trial)  # Write your machine learning logic here.
    return evaluation_score  # Return the evaluation score of the trained model.

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=N_TRIALS)  # Specify the number of trials. 
```



Пример № 1. Хотим минимизировать $(x-2)^2$. И хотя ответ для нас очевиден, Optuna не знает этого


In [ ]:
import optuna  # Remember to install optuna with `!pip install optuna` first.

def objective(trial):
    x = trial.suggest_uniform('x', -100, 100)
    return (x - 2) ** 2

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2023-05-10 19:43:23,107] A new study created in memory with name: no-name-d39e96f2-2532-4af0-a0f6-9e47fbe53bba
<ipython-input-2-dec494bddcc6>:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  x = trial.suggest_uniform('x', -100, 100)
[I 2023-05-10 19:43:23,113] Trial 0 finished with value: 390.01309103755943 and parameters: {'x': 21.74874910057747}. Best is trial 0 with value: 390.01309103755943.
[I 2023-05-10 19:43:23,116] Trial 1 finished with value: 7664.710179435414 and parameters: {'x': -85.54833053482753}. Best is trial 0 with value: 390.01309103755943.
[I 2023-05-10 19:43:23,120] Trial 2 finished with value: 4293.14632718253 and parameters: {'x': 67.52210563758257}. Best is trial 0 with value: 390.01309103755943.
[I 2023-05-10 19:43:23,123] Trial 3 finished with value: 1078.170536314684 and parameters: {'x': 

In [ ]:
print('Minimum objective value: ' + str(study.best_value))
print('Best parameter: ' + str(study.best_params))

Minimum objective value: 0.00010340779016253003
Best parameter: {'x': 1.9898310379014115}


В сухом остатке, вот что нужно для оптимизации:

- Определить `objective_function`, которая будет считать наш таргет, который мы хотим минимизировать или максимизировать
- Создать объект `study`
- Начать оптимизацию при помощи `study.optimize`, передав нужное число трайлов

Пример № 2. Наконец-то ML: линейная регрессия + Boston Hiuston Dataset

In [ ]:
!pip install scikit-learn==1.1  # for boston dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sklearn.datasets
import sklearn.linear_model
import sklearn.metrics

# hyperparameter setting
alpha = 1.0

# data loading and train-test split
X, y = sklearn.datasets.load_boston(return_X_y=True)
X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, random_state=0)

# model training and evaluation
model = sklearn.linear_model.Lasso(alpha=alpha)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
error = sklearn.metrics.mean_squared_error(y_val, y_pred)

# output: evaluation score
print('Mean squared error: ' + str(error))

Mean squared error: 36.63182007429979


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

Результат работы очень сильно зависит от константы для $l_1$-регуляризации, и для человека подбирать этот коэффициент утомительно. Мы можем сделать это с Optuna

In [ ]:
import optuna
import sklearn.datasets
import sklearn.linear_model
import sklearn.metrics

def objective(trial):
    # hyperparameter setting
    alpha = trial.suggest_uniform('alpha', 0.0, 2.0)
    
    # data loading and train-test split
    X, y = sklearn.datasets.load_boston(return_X_y=True)
    X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, random_state=0)
    
    # model training and evaluation
    model = sklearn.linear_model.Lasso(alpha=alpha)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    error = sklearn.metrics.mean_squared_error(y_val, y_pred)

    # output: evaluation score
    return error

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

[I 2023-05-10 19:48:02,991] A new study created in memory with name: no-name-5d2ef8aa-61b2-4478-b529-0252b780f3e5
<ipython-input-6-93844abcc3ce>:8: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  alpha = trial.suggest_uniform('alpha', 0.0, 2.0)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the data

In [ ]:
print('Minimum mean squared error: ' + str(study.best_value))
print('Best parameter: ' + str(study.best_params))

Minimum mean squared error: 29.81791243854349
Best parameter: {'alpha': 0.0013248445247513738}


In [ ]:
study.trials_dataframe()  # all results

,number,value,datetime_start,datetime_complete,duration,params_alpha,state
0,0,39.463367,2023-05-10 19:48:02.995239,2023-05-10 19:48:03.004142,0 days 00:00:00.008903,1.462557,COMPLETE
1,1,32.742175,2023-05-10 19:48:03.006363,2023-05-10 19:48:03.014197,0 days 00:00:00.007834,0.307288,COMPLETE
2,2,32.617123,2023-05-10 19:48:03.015726,2023-05-10 19:48:03.022410,0 days 00:00:00.006684,0.258297,COMPLETE
3,3,33.916796,2023-05-10 19:48:03.023800,2023-05-10 19:48:03.030381,0 days 00:00:00.006581,0.598318,COMPLETE
4,4,39.785019,2023-05-10 19:48:03.032614,2023-05-10 19:48:03.038518,0 days 00:00:00.005904,1.567133,COMPLETE
5,5,35.398217,2023-05-10 19:48:03.041078,2023-05-10 19:48:03.046758,0 days 00:00:00.005680,0.840180,COMPLETE
6,6,40.090171,2023-05-10 19:48:03.047933,2023-05-10 19:48:03.056070,0 days 00:00:00.008137,1.660663,COMPLETE
7,7,33.113633,2023-05-10 19:48:03.058668,2023-05-10 19:48:03.064495,0 days 00:00:00.005827,0.422055,COMPLETE
8,8,34.206528,2023-05-10 19:48:03.066997,2023-05-10 19:48:03.072757,0 days 00:00:00.005760,0.651551,COMPLETE
9,9,33.174720,2023-05-10 19:48:03.075258,2023-05-10 19:48:03.081124,0 days 00:00:00.005866,0.438191,COMPLETE


Усложняем пример - добавляем условий

In [ ]:
import optuna
import sklearn.datasets
import sklearn.linear_model
import sklearn.metrics

def objective(trial):
    # hyperparameter setting
    regression_method = trial.suggest_categorical('regression_method', ('ridge', 'lasso'))
    if regression_method == 'ridge':
        ridge_alpha = trial.suggest_uniform('ridge_alpha', 0.0, 2.0)
        model = sklearn.linear_model.Ridge(alpha=ridge_alpha)
    else:
        lasso_alpha = trial.suggest_uniform('lasso_alpha', 0.0, 2.0)
        model = sklearn.linear_model.Lasso(alpha=lasso_alpha)
    
    # data loading and train-test split
    X, y = sklearn.datasets.load_boston(return_X_y=True)
    X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, random_state=0)

    # model training and evaluation
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    error = sklearn.metrics.mean_squared_error(y_val, y_pred)
  
    # output: evaluation score
    return error

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

[I 2023-05-10 19:50:16,404] A new study created in memory with name: no-name-11719a29-4313-49d2-ab11-585f68a04451
<ipython-input-9-36caf4765894>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  ridge_alpha = trial.suggest_uniform('ridge_alpha', 0.0, 2.0)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can f

In [ ]:
print('Minimum mean squared error: ' + str(study.best_value))
print('Best parameter: ' + str(study.best_params))

study.trials_dataframe()

Minimum mean squared error: 29.78704756207891
Best parameter: {'regression_method': 'ridge', 'ridge_alpha': 0.004902589858662824}


,number,value,datetime_start,datetime_complete,duration,params_lasso_alpha,params_regression_method,params_ridge_alpha,state
0,0,29.807200,2023-05-10 19:50:28.289878,2023-05-10 19:50:28.313336,0 days 00:00:00.023458,NaN,ridge,0.025553,COMPLETE
1,1,30.821358,2023-05-10 19:50:28.318854,2023-05-10 19:50:28.333966,0 days 00:00:00.015112,NaN,ridge,1.842356,COMPLETE
2,2,30.224196,2023-05-10 19:50:28.342832,2023-05-10 19:50:28.353507,0 days 00:00:00.010675,NaN,ridge,0.521304,COMPLETE
3,3,30.772504,2023-05-10 19:50:28.355616,2023-05-10 19:50:28.367635,0 days 00:00:00.012019,NaN,ridge,1.680334,COMPLETE
4,4,30.156337,2023-05-10 19:50:28.378638,2023-05-10 19:50:28.390947,0 days 00:00:00.012309,NaN,ridge,0.427701,COMPLETE
5,5,36.388248,2023-05-10 19:50:28.392917,2023-05-10 19:50:28.402031,0 days 00:00:00.009114,0.970449,lasso,NaN,COMPLETE
6,6,36.146426,2023-05-10 19:50:28.404068,2023-05-10 19:50:28.420981,0 days 00:00:00.016913,0.940229,lasso,NaN,COMPLETE
7,7,40.587051,2023-05-10 19:50:28.440949,2023-05-10 19:50:28.454219,0 days 00:00:00.013270,1.803665,lasso,NaN,COMPLETE
8,8,33.025638,2023-05-10 19:50:28.461691,2023-05-10 19:50:28.471370,0 days 00:00:00.009679,0.397720,lasso,NaN,COMPLETE
9,9,35.871152,2023-05-10 19:50:28.473282,2023-05-10 19:50:28.482216,0 days 00:00:00.008934,0.904653,lasso,NaN,COMPLETE


Больше примеров [на официальном сайте](https://optuna.readthedocs.io)

### Примеры посложнее

[Optuna для Yolov5](https://github.com/savchenkoyana/yolov5/blob/master/train.py)